# Técnicas Computacionales Avanzadas de Hidro/Aero-Dinámica en Ingeniería y Astrofísica

## Elementos Finitos: Metodo Galerkin Discontinuo

**MSc Erick Urquilla, Universidad de Tennessee, Knoxville, USA**

Resolveremos las ecuaciones que gobiernan la dinámica de fluidos poco profundos incompresibles. El fluido está limitado inferiormente y a los costados por superficies rígidas y superiormente sin restricción. Estas ecuaciones se derivan de las ecuaciones de Navier-Stokes cuando el dominio espacial horizontal del fluido excede significativamente al vertical, manteniendo el equilibrio hidrostático y densidad constante:

$$
\frac{\partial h}{\partial t} + \frac{\partial}{\partial x}\left(hu\right) = 0
$$
$$
\frac{\partial}{\partial t}\left(hu\right) + \frac{\partial}{\partial x}\left(hu^2 + \frac{gh^2}{2}\right) = 0
$$

Aquí, $h$ es la altura del fluido, $u$ es la velocidad horizontal y $g$ es la aceleración de la gravedad. El movimiento del fluido está dictado exclusivamente por las ecuaciones de conservación de masa y momento.

### Importaremos los paquetes necesarios

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display
import time
import os
import glob
import imageio.v2 as imageio
import sys
from pathlib import Path

### Importando los scripts con el código de Galerkin discontinuo

In [ ]:
aqui_camino = Path.cwd()
cdg_fuente_camino = (aqui_camino / '..' / 'cdg_fuente').resolve()
sys.path.append(str(cdg_fuente_camino))

import bases
import galerkin_discontinuo 
import paso_de_tiempo
import graficos

### Parámetros de entrada

| Nombre de la variable | Unidades | Descripción |
|-----------------------|----------|-------------|
| **x_inicial**         | m        | Coordenada inicial del dominio espacial donde comienza la simulación. |
| **x_final**           | m        | Coordenada final del dominio espacial.|
| **N_elementos**       | —        | Número de elementos finitos en los que se subdivide el dominio espacial. Afecta la resolución espacial. |
| **N_nodos**           | —        | Número de nodos (o puntos de interpolación) dentro de cada elemento. Para simplicidad se asume `N_nodos ≥ 2`. |
| **n_pasos**           | —        | Número de pasos de tiempo que se ejecutarán en la simulación. Determina la resolución temporal. |
| **t_total**           | s        | Tiempo total de la simulación. |
| **n_nodos_cuadratura_gauss** | — | Número de nodos (puntos) utilizados en la cuadratura de Gauss para integrar numéricamente las funciones dentro de cada elemento. Un mayor número incrementa la precisión de la integración. |


In [ ]:
# Dominio espacial de la simulacion 
x_inicial = 0 # (m) cooordenda inicial del dominio
x_final = 10 # (m) coordenada final del dominio

# Parametros del metodo de elementos finitos
N_elementos = 6 # numero de elementos
N_nodos = 4 # numero de nodos por elemento (por simplicidad solo consideramos N_nodos >= 2)

# Dominio temporal 
n_pasos = 100 # numero de pasos temporales
t_total = 1 # (s) tiempo final

# Integracion numerica con cuadratura de Gauss
n_nodos_cuadratura_gauss = 20

### Generacion de malla y condiciones iniciales

In [ ]:
# Generando malla
longitud_elemento = (x_final - x_inicial) / N_elementos
malla = np.array([np.linspace(x_inicial + i * (x_final - x_inicial) / N_elementos, x_inicial + (i + 1) * (x_final - x_inicial) / N_elementos, N_nodos) for i in range(N_elementos)])      

# Condiciones iniciales
h = 1.0 + 0.1 * np.exp( - ( malla - 5.0 )**2 ) # Altura (m)
u = malla * 0.0 # Velocidad horizontal (m/s)

# creando carpeta para guardar las imagenes
os.makedirs('imagenes', exist_ok=True)
for f in glob.glob('imagenes/*.png'):
    os.remove(f)

graficos.plot_simulation(malla, h, u, N_elementos, 0, 0, display=True)

### Llamando funciones del metodo de Dicontinuos Galerkin

In [ ]:
cuadratura_de_gauss, pesos_de_gauss = np.polynomial.legendre.leggauss(n_nodos_cuadratura_gauss)
polinomios_de_lagrange_en_cuadratura_de_gauss, derivada_x_polinomios_de_lagrange_en_cuadratura_de_gauss = bases.generate_reference_space(malla, cuadratura_de_gauss)
matriz_de_masa_inversa = galerkin_discontinuo.calcula_inversa_matriz_de_masa(longitud_elemento, pesos_de_gauss, polinomios_de_lagrange_en_cuadratura_de_gauss, N_nodos)
matriz_de_rigidez = galerkin_discontinuo.calcula_matrix_de_rigidez(longitud_elemento, pesos_de_gauss, polinomios_de_lagrange_en_cuadratura_de_gauss, derivada_x_polinomios_de_lagrange_en_cuadratura_de_gauss)

### Evolucion temporal de la solucion: Método de Euler hacia adelante

In [ ]:
# paso de tiempo
paso_t = np.array(t_total/n_pasos) 

# evolucionando en el tiempo la ecuacion diferencial
for numero_de_paso in np.arange(n_pasos+1):

    print(f'paso {numero_de_paso}')

    graficos.plot_simulation(malla, h, u, N_elementos, paso_t, numero_de_paso, display=False)

    dh_dt, du_dt = paso_de_tiempo.compute_dhdt_du_dt(h, u, matriz_de_rigidez, matriz_de_masa_inversa)

    h += dh_dt * paso_t
    u += du_dt * paso_t

print(f'Done')

### Graficando la solución

In [ ]:
duracion_por_cuadro = 0.3  # segundos

files = sorted(glob.glob("./imagenes/plt*.png"))
files = sorted(files, key=lambda x: int(os.path.splitext(os.path.basename(x))[0][3:]))

frames = [imageio.imread(f) for f in files]
imageio.mimsave("imagenes/animacion.gif", frames, duration=duracion_por_cuadro)

display(Image(filename="imagenes/animacion.gif"))